In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
!git clone https://github.com/SeanNaren/warp-ctc.git

Cloning into 'warp-ctc'...
remote: Enumerating objects: 829, done.
remote: Total 829 (delta 0), reused 0 (delta 0), pack-reused 829
Receiving objects: 100% (829/829), 388.85 KiB | 140.00 KiB/s, done.
Resolving deltas: 100% (419/419), done.
Checking connectivity... done.


In [4]:
%cd warp-ctc

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc


In [5]:
mkdir -p build

In [6]:
cd build

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/build


In [7]:
!cmake ..

-- The C compiler identification is GNU 5.4.0
-- The CXX compiler identification is GNU 5.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found suit

In [8]:
!make -j

[ 11%] Building NVCC (Device) object CMakeFiles/warpctc.dir/src/warpctc_generated_reduce.cu.o
[ 22%] Building NVCC (Device) object CMakeFiles/warpctc.dir/src/warpctc_generated_ctc_entrypoint.cu.o
Scanning dependencies of target warpctc
[ 33%] Linking CXX shared library libwarpctc.so
[ 33%] Built target warpctc
[ 44%] Building NVCC (Device) object CMakeFiles/test_gpu.dir/tests/test_gpu_generated_test_gpu.cu.o
Scanning dependencies of target test_cpu
[ 55%] Building CXX object CMakeFiles/test_cpu.dir/tests/test_cpu.cpp.o
[ 66%] Building CXX object CMakeFiles/test_cpu.dir/tests/random.cpp.o
[ 77%] Linking CXX executable test_cpu
[ 77%] Built target test_cpu
Scanning dependencies of target test_gpu
[ 88%] Building CXX object CMakeFiles/test_gpu.dir/tests/random.cpp.o
[100%] Linking CXX executable test_gpu
[100%] Built target test_gpu


In [10]:
cd ..

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc


In [11]:
cd pytorch_binding

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/pytorch_binding


In [12]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating warpctc_pytorch.egg-info
writing warpctc_pytorch.egg-info/PKG-INFO
writing dependency_links to warpctc_pytorch.egg-info/dependency_links.txt
writing top-level names to warpctc_pytorch.egg-info/top_level.txt
writing manifest file 'warpctc_pytorch.egg-info/SOURCES.txt'
writing manifest file 'warpctc_pytorch.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib.linux-x86_64-3.9
creating build/lib.linux-x86_64-3.9/warpctc_pytorch
copying warpctc_pytorch/__init__.py -> build/lib.linux-x86_64-3.9/warpctc_pytorch
running build_ext
building 'warpctc_pytorch._warp_ctc' extension
creating /workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/pytorch_binding/build/temp.linux-x86_64-3.9
creating /workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc/pytorch_binding/build/temp.linux-x86_64-3.9/src
Emitting ninja build file /workspace/zha

In [13]:
!python -V

Python 3.9.5


In [14]:
cd ..

/workspace/zhanghui/DeepSpeech-2.x/docs/topic/ctc/warp-ctc


In [15]:
import torch
import torch.nn as nn
import warpctc_pytorch as wp
import paddle.nn as pn
import paddle

grep: warning: GREP_OPTIONS is deprecated; please use an alias or script


In [16]:
torch.__version__

'1.10.0+cu102'

In [17]:
paddle.__version__

'2.2.0'

In [18]:
probs = torch.FloatTensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]
    ]]).transpose(0, 1).contiguous()
print(probs.size())
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)
bs = probs.size(1)

ctc_loss = wp.CTCLoss(size_average=False, length_average=False)
cost = ctc_loss(probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

torch.Size([2, 1, 5])
2.4628584384918213
[[[ 0.17703122 -0.70812464  0.17703122  0.17703122  0.17703122]]

 [[ 0.17703122  0.17703122 -0.70812464  0.17703122  0.17703122]]]


In [19]:
probs = torch.FloatTensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1]
    ]]).transpose(0, 1).contiguous()
labels = torch.IntTensor([1, 2])
label_sizes = torch.IntTensor([2])
probs_sizes = torch.IntTensor([2])
probs.requires_grad_(True)
bs = probs.size(1)

log_probs = torch.log_softmax(probs, axis=-1)

ctc_loss1 = nn.CTCLoss(reduction='none')
cost = ctc_loss1(log_probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

2.4628584384918213
[[[ 0.1770312 -0.7081248  0.1770312  0.1770312  0.1770312]]

 [[ 0.1770312  0.1770312 -0.7081248  0.1770312  0.1770312]]]


In [22]:
paddle.set_device('cpu')
probs = paddle.to_tensor([[
        [0.1, 0.6, 0.1, 0.1, 0.1], [0.1, 0.1, 0.6, 0.1, 0.1],
    ]]).transpose([1,0,2])
print(probs.shape) # (T, B, D)
labels = paddle.to_tensor([[1, 2]], dtype='int32') #（B，L)
print(labels.shape)
label_sizes = paddle.to_tensor([2], dtype='int64')
probs_sizes = paddle.to_tensor([2], dtype='int64')
bs = paddle.shape(probs)[1]
probs.stop_gradient=False

ctc_loss = pn.CTCLoss(reduction='none')
cost = ctc_loss(probs, labels, probs_sizes, label_sizes)
cost = cost.sum() / bs
print(cost.item())
cost.backward()
print(probs.grad.numpy())

[2, 1, 5]
[1, 2]
2.4628584384918213
[[[ 0.17703122 -0.70812464  0.17703122  0.17703122  0.17703122]]

 [[ 0.17703122  0.17703122 -0.70812464  0.17703122  0.17703122]]]
